# About This Notebook

This implementation is based on a vanilla **swin_large_patch4_window12_384** in Pytorch for the Pawpularity Competition.  
This model uses **both images and dense features** for score prediction.  
**This scores around 18.1 LB.**

Training Params: -
1. **Dataset**: - 3채널 RGB 이미지 (384x384) 
2. **Augmentations**: - Resize, Normalize, HorizontalFlip, VerticalFlip, RandomBrightness, RandomResizedCrop, HueSaturationValue, RandomBrightnessContrast
3. **Optimizer**: - AdamW
4. **Scheduler**: - CosineAnnealingLR
5. **Model**: - swin_small_patch4_window12_384 (원래 swin_large였는데 small로 바꿈)
6. **Initial Weights**: - Imagenet ??
5. **Max Epochs**: - 8 (~23 min per epoch on P100 PCIE GPU)
6. **Saved Weights**: - 10-fold ensemble. Weights having highest OOF score on RMSE metric were saved.


# Get GPU Info

In [1]:
!nvidia-smi

Tue Oct 26 01:23:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -qq timm
!pip install -qq albumentations==1.0.3
!pip install -qq grad-cam
!pip install -qq ttach

     |████████████████████████████████| 376 kB 5.3 MB/s 
     |████████████████████████████████| 98 kB 3.7 MB/s 
     |████████████████████████████████| 47.6 MB 36 kB/s 
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [3]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [7]:
import os
os.chdir("/content/drive/MyDrive/0_Coding/K-digital/pj2")
!pwd

/content/drive/MyDrive/0_Coding/K-digital/pj2


In [5]:
import sys
sys.path.append("/content/drive/MyDrive/0_Coding/K-digital/pj2")

In [8]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os
import glob
import random
import cv2
pd.set_option('display.max_columns', None)

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

Using device: cuda


In [22]:
csv_dir = './'          # CSV가 있는 폴더 입력


train_dir = './train/'  # 사진이 있는 폴더 입력
test_dir = './test/'    

models_dir = './models' # 모델이 들어가 있는 폴더 입력

test_file_path = os.path.join(csv_dir, 'test.csv')                    # 폴더와 파일명 입력
# sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv') # 이건 kaggle 용
print(f'Test file: {test_file_path}')
print(f'Models path: {models_dir}')

Test file: ./test.csv
Models path: ./models


In [37]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv(test_file_path)
sample_df = pd.read_csv(sample_sub_file_path)

In [24]:
def return_filpath(name, folder):
    path = os.path.join(folder, f'{name}.jpg')
    return path

In [38]:
train_df['image_path'] = train_df['Id'].apply(lambda x: return_filpath(x, folder=train_dir))

In [25]:
test_df['image_path'] = test_df['Id'].apply(lambda x: return_filpath(x, folder=test_dir))

In [39]:
train_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,image_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,./train/0007de18844b0dbbb5e1f607da0606e0.jpg
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,./train/0009c66b9439883ba2750fb825e1d7db.jpg
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,./train/0013fd999caf9a3efe1352ca1b0d937e.jpg
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,./train/0018df346ac9c1d8413cfcc888ca8246.jpg
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,./train/001dc955e10590d3ca4673f034feeef2.jpg


In [96]:
train_ = train_df[5:20]
train_

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,image_path
5,001dd4f6fafb890610b1635f967ea081,0,0,1,0,0,0,0,0,0,0,0,1,74,./train/001dd4f6fafb890610b1635f967ea081.jpg
6,0023b8a3abc93c712edd6120867deb53,0,1,1,1,0,0,0,0,1,1,0,0,22,./train/0023b8a3abc93c712edd6120867deb53.jpg
7,0031d6a9ef7340f898c3e05f92c7bb04,0,1,1,0,0,0,1,1,0,0,1,0,35,./train/0031d6a9ef7340f898c3e05f92c7bb04.jpg
8,0042bc5bada6d1cf8951f8f9f0d399fa,0,1,1,1,0,0,0,0,0,0,0,0,53,./train/0042bc5bada6d1cf8951f8f9f0d399fa.jpg
9,0049cb81313c94fa007286e9039af910,0,1,1,1,0,0,0,0,0,0,0,0,21,./train/0049cb81313c94fa007286e9039af910.jpg
10,005017716086b8d5e118dd9fe26459b1,0,1,1,1,0,0,0,0,0,0,0,0,28,./train/005017716086b8d5e118dd9fe26459b1.jpg
11,00524dbf2637a80cbc80f70d3ff59616,0,1,1,1,0,0,0,0,0,0,0,0,2,./train/00524dbf2637a80cbc80f70d3ff59616.jpg
12,00630b1262efe301cb15a3b2022ba744,0,1,1,1,0,0,0,0,0,0,0,0,18,./train/00630b1262efe301cb15a3b2022ba744.jpg
13,006483b96ca9c09b7afed3e3d3af539d,0,1,1,1,0,0,0,0,1,1,0,0,41,./train/006483b96ca9c09b7afed3e3d3af539d.jpg
14,00655425c10d4c082dd7eeb97fa4fb17,0,1,0,0,0,1,0,0,0,0,0,0,13,./train/00655425c10d4c082dd7eeb97fa4fb17.jpg


# CFG

In [100]:
params = {
    'model': 'vit_small_patch32_384',
    'dense_features': ['Subject Focus', 'Eyes', 'Face', 'Near',
                       'Action', 'Accessory', 'Group', 'Collage',
                       'Human', 'Occlusion', 'Info', 'Blur'],
    'pretrained': False,
    'inp_channels': 3,
    'im_size': 384,
    'device': device,
    'batch_size': 16,
    'num_workers' : 2,
    'out_features': 1,
    'debug': False
}

In [101]:
# if params['debug']:
#     test_df = test_df.sample(frac=0.1)

if params['debug']:
    train_ = train_.sample(frac=0.1)

# Augmentations

In [102]:
def get_test_transforms(DIM = params['im_size']):
    return albumentations.Compose(
        [
          albumentations.Resize(DIM,DIM),
          albumentations.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225],
          ),
          ToTensorV2(p=1.0)
        ]
    )

# Dataset

In [103]:
class CuteDataset(Dataset):
    def __init__(self, images_filepaths, dense_features, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.dense_features = dense_features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        dense = self.dense_features[idx, :]
        label = torch.tensor(self.targets[idx]).float()
        return image, dense, label

# CNN Model

In [104]:
class PetNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'], inp_channels=params['inp_channels'],
                 pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 128)
        self.fc = nn.Sequential(
            nn.Linear(128 + num_dense, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        output = self.fc(x)
        return output

# Prediction

In [157]:
for i in range(40,80, 8): 
    print(i, i+7)

40 47
48 55
56 63
64 71
72 79


In [161]:
paw = []
id_ = []
origin_paw = []
for i in range(40,80, 8):                # 최대 8개만 넣을 수 있어서 7개로 제한 
    train_ = train_df[i:i+7]            # 전체가 돌아갈 수 있도록 최대값 수정
    for vl in train_[['Id', 'Pawpularity']].values:
        id_.append(vl[0])
        origin_paw.append(vl[1])
    predicted_labels = None
    for model_name in glob.glob(models_dir + '/*.pth'):                # 여기 폴더에 모델을 여러게 넣어서 사용할 수 있음.
        model = PetNet()
        model.load_state_dict(torch.load(model_name))
        model = model.to(params['device'])
        model.eval()

        test_dataset = CuteDataset(
            images_filepaths = train_['image_path'].values,
            dense_features = train_[params['dense_features']].values,
            targets = train_['Pawpularity'].values,
            transform = get_test_transforms()
        )
        test_loader = DataLoader(
            test_dataset, batch_size=params['batch_size'],
            shuffle=False, num_workers=params['num_workers'],
            pin_memory=True
        )

        temp_preds = None
        with torch.no_grad():
            for (images, dense, target) in tqdm(test_loader, desc=f'Predicting. '):
                images = images.to(params['device'], non_blocking=True)
                dense = dense.to(params['device'], non_blocking=True)
                predictions = torch.sigmoid(model(images, dense)).to('cpu').numpy()*100
                
                if temp_preds is None:
                    temp_preds = predictions
                else:
                    temp_preds = np.vstack((temp_preds, predictions))

        if predicted_labels is None:
            predicted_labels = temp_preds
        else:
            predicted_labels += temp_preds
   
    predicted_labels /= (len(glob.glob(models_dir + '/*.pth')))
    for j in predicted_labels:
        paw.append(j[0])


# DataFrame
sub_df = pd.DataFrame()
sub_df['Id'] = id_
sub_df['origin'] = origin_paw
sub_df['predict'] = paw

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
   

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

...

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
   

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

Predicting. :   0%|          | 0/1 [00:00<?, ?it/s]

In [162]:
sub_df

,Id,origin,predict
0,00f6f38380bec413074150c18b2828d2,16,28.149458
1,00fb5be5bf4e956ea4abb99f41c0b9ec,42,32.631954
2,00fe27dcd1daf42c85365e837415878c,53,41.526848
3,00ff89af70382e1dede4c7091c454e47,31,26.994959
4,0117f444e6118619c4f6492cba703177,95,39.384068
5,012518ff91b2484618693a277c0b07d5,34,36.309776
6,0126fe11deba78f84fd05cd5f12c5a4b,37,32.667046
7,01318357ff47c204f0d5c5a28e4eac77,47,33.195072
8,01320990ba35993ad92b22f2d2ef039b,62,54.956486
9,013f86ed0e765b189990d3d5ac28bd7d,100,44.697918


In [163]:
sub_df[['origin','predict']].corr()

,origin,predict
origin,1.000000,0.508583
predict,0.508583,1.000000


In [152]:
# sub_df = pd.DataFrame()
# sub_df['Id'] = id_
# sub_df['origin'] = origin_paw
# sub_df['Pawpularity'] = paw

In [153]:
sub_df

,Id,origin,Pawpularity
0,0009c66b9439883ba2750fb825e1d7db,42,[26.535645]
1,0013fd999caf9a3efe1352ca1b0d937e,28,[47.411026]
2,0018df346ac9c1d8413cfcc888ca8246,15,[49.165756]
3,001dc955e10590d3ca4673f034feeef2,72,[30.394062]
4,001dd4f6fafb890610b1635f967ea081,74,[57.02397]
5,0023b8a3abc93c712edd6120867deb53,22,[34.041733]
6,0031d6a9ef7340f898c3e05f92c7bb04,35,[39.5295]
7,0049cb81313c94fa007286e9039af910,21,[28.757242]
8,005017716086b8d5e118dd9fe26459b1,28,[35.98075]
9,00524dbf2637a80cbc80f70d3ff59616,2,[39.672234]
